In [1]:
import pickle
import numpy as np
from sklearn.preprocessing import RobustScaler,MinMaxScaler
X_ppg,X_qual,X_acc,y_rr, y_respiration,y_inspiration,y_expiration,groups,X_respiration,y_activity,y_label = pickle.load(open('../../affect-recognition/data/tabular_data_8.p','rb'))

In [2]:
from sklearn.preprocessing import OneHotEncoder
index = np.where((y_activity!=0)&(y_activity!=3)&(y_activity!=6)&(y_activity!=8)&(y_activity!=2))[0]
X_acc = X_acc[index]
y_activity = y_activity[index]
groups = groups[index]
y_activity = OneHotEncoder().fit_transform(y_activity.reshape(-1,1)).todense()


/home/azim/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [4]:
from keras.layers import Input, LSTM, RepeatVector,Bidirectional,Multiply,multiply,Permute
from keras.layers import TimeDistributed,Dense,Flatten,Reshape,Lambda,Activation,GRU,Conv1D,MaxPool1D,Dropout,BatchNormalization
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import numpy as np
import keras.backend as K
from keras import metrics,losses
import tensorflow as tf
import tensorflow_probability as tf
from sklearn.model_selection import GroupKFold

def get_base_CNN(timesteps_ppg=256,input_dim_ppg=3,first_dim=60,
                 optimizer='adam',loss='kullback_leibler_divergence',output_dim=4,
                 metrics = ['accuracy']):
    inputs = Input(shape=(timesteps_ppg, input_dim_ppg))
    fe = Conv1D(100,4, padding='same',activation='tanh')(inputs)
    fe = MaxPool1D(2)(fe)
    fe = BatchNormalization()(fe)
    fe = Dropout(.2)(fe)
    fe = Conv1D(100,4, padding='same',activation='tanh')(fe)
    fe = MaxPool1D(4)(fe)
    fe = BatchNormalization()(fe)
    fe = Dropout(.2)(fe)
    fe = Conv1D(50,4, padding='same',activation='tanh')(fe)
    fe = MaxPool1D(8)(fe)
    fe = BatchNormalization()(fe)
    fe = Dropout(.2)(fe)
    flattened = Flatten()(fe)
    output = Dense(30,activation='relu')(flattened)
    output = Dense(output_dim,activation='sigmoid')(output)
    sequence_autoencoder = Model(inputs=[inputs], outputs=[output])
    sequence_autoencoder.compile(optimizer=optimizer,loss=loss,metrics=[metrics])
    return sequence_autoencoder


gkf = GroupKFold(n_splits=len(np.unique(groups)))
indexes = list(gkf.split(X_acc,groups=groups))
for train,test in indexes:
    X_acc_train,X_acc_test = X_acc[train],X_acc[test]
    y_activity_train,y_activity_test = y_activity[train],y_activity[test]
    y_activity_train.shape
    sequence_autoencoder = get_base_CNN()
    sequence_autoencoder.summary()
    filepath = '../../affect-recognition/data/models/activity/'+str(groups[test][0])+'.h5'
    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True,save_weights_only=False, mode='min')
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=30)
    callbacks_list = [es,checkpoint]
    history = sequence_autoencoder.fit(X_acc_train, y_activity_train,
                    epochs=400,
                    batch_size=500,
                    shuffle=True,
                    validation_split=.3,callbacks=callbacks_list)
    %matplotlib notebook
    import matplotlib.pyplot as plt
    from sklearn.metrics import confusion_matrix
    from keras.models import load_model
    sequence_autoencoder.load_weights(filepath)
    y_activity_pred = sequence_autoencoder.predict(X_acc_test)
    print(confusion_matrix(np.int64(np.argmax(y_activity_test,axis=1)),np.int64(np.argmax(y_activity_pred,axis=1))))

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 256, 3)            0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 256, 100)          1300      
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 128, 100)          0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 128, 100)          400       
_________________________________________________________________
dropout_4 (Dropout)          (None, 128, 100)          0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 128, 100)          40100     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 32, 100)           0   

12623/12623 [==============================] - 3s 228us/step - loss: 1.4592e-04 - accuracy: 0.8809 - val_loss: 0.0766 - val_accuracy: 0.5075

Epoch 00025: val_loss improved from 0.09206 to 0.07665, saving model to ../../affect-recognition/data/models/activity/8.h5
Epoch 26/400
12623/12623 [==============================] - 3s 226us/step - loss: 1.4046e-04 - accuracy: 0.8805 - val_loss: 0.0581 - val_accuracy: 0.4700

Epoch 00026: val_loss improved from 0.07665 to 0.05811, saving model to ../../affect-recognition/data/models/activity/8.h5
Epoch 27/400
12623/12623 [==============================] - 3s 228us/step - loss: 1.3421e-04 - accuracy: 0.8815 - val_loss: 0.0373 - val_accuracy: 0.6607

Epoch 00027: val_loss improved from 0.05811 to 0.03727, saving model to ../../affect-recognition/data/models/activity/8.h5
Epoch 28/400
12623/12623 [==============================] - 3s 227us/step - loss: 1.2203e-04 - accuracy: 0.8781 - val_loss: 0.0267 - val_accuracy: 0.7561

Epoch 00028: val_loss im

12709/12709 [==============================] - 3s 263us/step - loss: 0.2383 - accuracy: 0.6746 - val_loss: 0.5915 - val_accuracy: 0.2302

Epoch 00001: val_loss improved from inf to 0.59154, saving model to ../../affect-recognition/data/models/activity/4.h5
Epoch 2/400
12709/12709 [==============================] - 3s 229us/step - loss: 0.0430 - accuracy: 0.7624 - val_loss: 0.5278 - val_accuracy: 0.2302

Epoch 00002: val_loss improved from 0.59154 to 0.52777, saving model to ../../affect-recognition/data/models/activity/4.h5
Epoch 3/400
12709/12709 [==============================] - 3s 231us/step - loss: 0.0187 - accuracy: 0.7776 - val_loss: 0.5141 - val_accuracy: 0.2302

Epoch 00003: val_loss improved from 0.52777 to 0.51407, saving model to ../../affect-recognition/data/models/activity/4.h5
Epoch 4/400
12709/12709 [==============================] - 3s 226us/step - loss: 0.0118 - accuracy: 0.7908 - val_loss: 0.5401 - val_accuracy: 0.2302

Epoch 00004: val_loss did not improve from 0.51

12709/12709 [==============================] - 3s 227us/step - loss: 1.5828e-04 - accuracy: 0.7864 - val_loss: 2.7032e-04 - val_accuracy: 0.6187

Epoch 00036: val_loss improved from 0.00048 to 0.00027, saving model to ../../affect-recognition/data/models/activity/4.h5
Epoch 37/400
12709/12709 [==============================] - 3s 226us/step - loss: 1.2529e-04 - accuracy: 0.7842 - val_loss: 3.5240e-04 - val_accuracy: 0.6211

Epoch 00037: val_loss did not improve from 0.00027
Epoch 38/400
12709/12709 [==============================] - 3s 225us/step - loss: 1.1398e-04 - accuracy: 0.7843 - val_loss: 4.8216e-04 - val_accuracy: 0.6198

Epoch 00038: val_loss did not improve from 0.00027
Epoch 39/400
12709/12709 [==============================] - 3s 220us/step - loss: 1.2848e-04 - accuracy: 0.7842 - val_loss: 4.6509e-04 - val_accuracy: 0.6240

Epoch 00039: val_loss did not improve from 0.00027
Epoch 40/400
12709/12709 [==============================] - 3s 228us/step - loss: 1.2617e-04 - accura

12709/12709 [==============================] - 3s 223us/step - loss: 2.7481e-05 - accuracy: 0.7754 - val_loss: 2.0564e-04 - val_accuracy: 0.6257

Epoch 00072: val_loss did not improve from 0.00004
Epoch 73/400
12709/12709 [==============================] - 3s 221us/step - loss: 2.4978e-05 - accuracy: 0.7700 - val_loss: 1.7654e-04 - val_accuracy: 0.6258

Epoch 00073: val_loss did not improve from 0.00004
Epoch 74/400
12709/12709 [==============================] - 3s 220us/step - loss: 2.3347e-05 - accuracy: 0.7745 - val_loss: 1.4820e-04 - val_accuracy: 0.6242

Epoch 00074: val_loss did not improve from 0.00004
Epoch 75/400
12709/12709 [==============================] - 3s 221us/step - loss: 2.5496e-05 - accuracy: 0.7721 - val_loss: 1.9748e-04 - val_accuracy: 0.6209

Epoch 00075: val_loss did not improve from 0.00004
Epoch 76/400
12709/12709 [==============================] - 3s 222us/step - loss: 2.3319e-05 - accuracy: 0.7717 - val_loss: 1.7871e-04 - val_accuracy: 0.6191

Epoch 00076: v

...

12915/12915 [==============================] - 3s 220us/step - loss: -4.7162e-06 - accuracy: 0.6098 - val_loss: 5.2745e-07 - val_accuracy: 0.5153

Epoch 00383: val_loss improved from 0.00000 to 0.00000, saving model to ../../affect-recognition/data/models/activity/11.h5
Epoch 384/400
12915/12915 [==============================] - 3s 224us/step - loss: -4.7237e-06 - accuracy: 0.6037 - val_loss: 4.5139e-07 - val_accuracy: 0.5124

Epoch 00384: val_loss improved from 0.00000 to 0.00000, saving model to ../../affect-recognition/data/models/activity/11.h5
Epoch 385/400
12915/12915 [==============================] - 3s 222us/step - loss: -4.7126e-06 - accuracy: 0.6043 - val_loss: 4.8572e-07 - val_accuracy: 0.5097

Epoch 00385: val_loss did not improve from 0.00000
Epoch 386/400
12915/12915 [==============================] - 3s 222us/step - loss: -4.7150e-06 - accuracy: 0.6033 - val_loss: 3.0351e-07 - val_accuracy: 0.5162

Epoch 00386: val_loss improved from 0.00000 to 0.00000, saving model to

12936/12936 [==============================] - 3s 218us/step - loss: 0.0044 - accuracy: 0.7693 - val_loss: 0.5560 - val_accuracy: 0.3364

Epoch 00004: val_loss improved from 0.56269 to 0.55599, saving model to ../../affect-recognition/data/models/activity/6.h5
Epoch 5/400
12936/12936 [==============================] - 3s 220us/step - loss: 0.0029 - accuracy: 0.7608 - val_loss: 0.5474 - val_accuracy: 0.3364

Epoch 00005: val_loss improved from 0.55599 to 0.54737, saving model to ../../affect-recognition/data/models/activity/6.h5
Epoch 6/400
12936/12936 [==============================] - 3s 215us/step - loss: 0.0023 - accuracy: 0.7588 - val_loss: 0.5377 - val_accuracy: 0.3360

Epoch 00006: val_loss improved from 0.54737 to 0.53768, saving model to ../../affect-recognition/data/models/activity/6.h5
Epoch 7/400
12936/12936 [==============================] - 3s 219us/step - loss: 0.0016 - accuracy: 0.7604 - val_loss: 0.5231 - val_accuracy: 0.3424

Epoch 00007: val_loss improved from 0.53768

12936/12936 [==============================] - 3s 220us/step - loss: 1.7720e-05 - accuracy: 0.7686 - val_loss: 1.2901e-04 - val_accuracy: 0.5909

Epoch 00068: val_loss did not improve from 0.00013
Epoch 69/400
12936/12936 [==============================] - 3s 218us/step - loss: 1.8410e-05 - accuracy: 0.7689 - val_loss: 1.2183e-04 - val_accuracy: 0.5904

Epoch 00069: val_loss improved from 0.00013 to 0.00012, saving model to ../../affect-recognition/data/models/activity/6.h5
Epoch 70/400
12936/12936 [==============================] - 3s 218us/step - loss: 1.3787e-05 - accuracy: 0.7681 - val_loss: 1.1262e-04 - val_accuracy: 0.5934

Epoch 00070: val_loss improved from 0.00012 to 0.00011, saving model to ../../affect-recognition/data/models/activity/6.h5
Epoch 71/400
12936/12936 [==============================] - 3s 220us/step - loss: 1.5082e-05 - accuracy: 0.7675 - val_loss: 9.9900e-05 - val_accuracy: 0.5925

Epoch 00071: val_loss improved from 0.00011 to 0.00010, saving model to ../../af

12936/12936 [==============================] - 3s 219us/step - loss: -4.0214e-07 - accuracy: 0.7826 - val_loss: 2.7473e-05 - val_accuracy: 0.5711

Epoch 00136: val_loss did not improve from 0.00002
Epoch 137/400
12936/12936 [==============================] - 3s 220us/step - loss: -6.0732e-07 - accuracy: 0.7818 - val_loss: 2.8101e-05 - val_accuracy: 0.5685

Epoch 00137: val_loss did not improve from 0.00002
Epoch 138/400
12936/12936 [==============================] - 3s 222us/step - loss: -5.3159e-07 - accuracy: 0.7808 - val_loss: 2.3659e-05 - val_accuracy: 0.5684

Epoch 00138: val_loss improved from 0.00002 to 0.00002, saving model to ../../affect-recognition/data/models/activity/6.h5
Epoch 139/400
12936/12936 [==============================] - 3s 221us/step - loss: -4.9753e-07 - accuracy: 0.7801 - val_loss: 2.6322e-05 - val_accuracy: 0.5714

Epoch 00139: val_loss did not improve from 0.00002
Epoch 140/400
12936/12936 [==============================] - 3s 219us/step - loss: -3.7201e-07

12936/12936 [==============================] - 3s 221us/step - loss: -2.7973e-06 - accuracy: 0.7840 - val_loss: 1.1592e-05 - val_accuracy: 0.5694

Epoch 00170: val_loss improved from 0.00001 to 0.00001, saving model to ../../affect-recognition/data/models/activity/6.h5
Epoch 171/400
12936/12936 [==============================] - 3s 220us/step - loss: -2.6088e-06 - accuracy: 0.7823 - val_loss: 1.1936e-05 - val_accuracy: 0.5691

Epoch 00171: val_loss did not improve from 0.00001
Epoch 172/400
12936/12936 [==============================] - 3s 221us/step - loss: -1.8749e-06 - accuracy: 0.7865 - val_loss: 8.4700e-06 - val_accuracy: 0.5747

Epoch 00172: val_loss improved from 0.00001 to 0.00001, saving model to ../../affect-recognition/data/models/activity/6.h5
Epoch 173/400
12936/12936 [==============================] - 3s 222us/step - loss: -2.6773e-06 - accuracy: 0.7851 - val_loss: 7.8965e-06 - val_accuracy: 0.5732

Epoch 00173: val_loss improved from 0.00001 to 0.00001, saving model to .

In [ ]:
def get_base_GRU(timesteps_ppg=256,input_dim_ppg=3,first_dim=60,
                 optimizer='adam',loss='binary_crossentropy',output_dim=9,
                 metrics = ['accuracy']):
    inputs = Input(shape=(timesteps_ppg, input_dim_ppg))
    encoded = Bidirectional(GRU(first_dim,return_sequences=True,activation='tanh',go_backwards=True))(inputs)
    dense_layer = TimeDistributed(Dense(1,activation='tanh'))(encoded)
    flattened = Flatten()(dense_layer)
    output = Dense(30,activation='relu')(flattened)
    output = Dense(output_dim,activation='sigmoid')(output)
    sequence_autoencoder = Model(inputs=[inputs], outputs=[output])
    sequence_autoencoder.compile(optimizer=optimizer,loss=loss,metrics=[metrics])
    return sequence_autoencoder


def get_base_CNN(timesteps_ppg=256,input_dim_ppg=3,first_dim=60,
                 optimizer='adam',loss='kullback_leibler_divergence',output_dim=4,
                 metrics = ['accuracy']):
    inputs = Input(shape=(timesteps_ppg, input_dim_ppg))
    fe = Conv1D(100,4, padding='same',activation='tanh')(inputs)
    fe = MaxPool1D(2)(fe)
    fe = BatchNormalization()(fe)
    fe = Dropout(.2)(fe)
    fe = Conv1D(100,4, padding='same',activation='tanh')(fe)
    fe = MaxPool1D(4)(fe)
    fe = BatchNormalization()(fe)
    fe = Dropout(.2)(fe)
    fe = Conv1D(50,4, padding='same',activation='tanh')(fe)
    fe = MaxPool1D(8)(fe)
    fe = BatchNormalization()(fe)
    fe = Dropout(.2)(fe)
    flattened = Flatten()(fe)
    output = Dense(30,activation='relu')(flattened)
    output = Dense(output_dim,activation='sigmoid')(output)
    sequence_autoencoder = Model(inputs=[inputs], outputs=[output])
    sequence_autoencoder.compile(optimizer=optimizer,loss=loss,metrics=[metrics])
    return sequence_autoencoder

# timesteps_ppg = 256
# input_dim_ppg = 3
# sequence_autoencoder = get_base_autoencoder(timesteps_ppg=timesteps_ppg,input_dim_ppg=input_dim_ppg,
#                                     first_dim=60,optimizer='adam',loss='mse')


In [ ]:
plt.figure(figsize=(10,5))
plt.plot(X_acc_test[20])
# plt.plot(X_ppg_pred[330],'r')
plt.show()

In [ ]:
y_rr_pred = sequence_autoencoder.predict(X_ppg_test)

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(y_rr_pred)
plt.plot(y_rr_test,'r')

In [ ]:
differences = np.abs(y_rr_pred-y_rr_test)

In [ ]:
relative_powers = [np.mean(a[:,-1]) for a in X_qual_test]

In [ ]:
plt.figure(figsize=(10,5))
plt.scatter(relative_powers,differences)
plt.show()

In [ ]:
np.mean(differences)